*   Nama : **Virgiawan Teguh Kusuma**
*   Email : **181240000833@unisnu.ac.id**
*   Domisili : **Kabupaten Jepara, Jawa Tengah**

In [ ]:
# import pandas dan membuat dataframe
import pandas as pd
dataframe = pd.read_csv('bbc-news-data.csv', sep='\t')

In [ ]:
dataframe

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [ ]:
dataframe = dataframe.drop(columns=['filename', 'content'])
dataframe

,category,title
0,business,Ad sales boost Time Warner profit
1,business,Dollar gains on Greenspan speech
2,business,Yukos unit buyer faces loan claim
3,business,High fuel prices hit BA's profits
4,business,Pernod takeover talk lifts Domecq
...,...,...
2220,tech,BT program to beat dialler scams
2221,tech,Spam e-mails tempt net shoppers
2222,tech,Be careful how you code
2223,tech,US cyber security chief resigns


In [ ]:
# one hot encoding
category = pd.get_dummies(dataframe.category)
newDataframe = pd.concat([dataframe, category], axis=1)
newDataframe = newDataframe.drop(columns=['category'])

In [ ]:
newDataframe

,title,business,entertainment,politics,sport,tech
0,Ad sales boost Time Warner profit,1,0,0,0,0
1,Dollar gains on Greenspan speech,1,0,0,0,0
2,Yukos unit buyer faces loan claim,1,0,0,0,0
3,High fuel prices hit BA's profits,1,0,0,0,0
4,Pernod takeover talk lifts Domecq,1,0,0,0,0
...,...,...,...,...,...,...
2220,BT program to beat dialler scams,0,0,0,0,1
2221,Spam e-mails tempt net shoppers,0,0,0,0,1
2222,Be careful how you code,0,0,0,0,1
2223,US cyber security chief resigns,0,0,0,0,1


In [ ]:
# mengubah nilai dari dataframe ke array
title = newDataframe['title'].values
label = newDataframe[newDataframe.columns[1:6]].values

In [ ]:
# split data training dan testing
from sklearn.model_selection import train_test_split
title_train, title_test, label_train, label_test = train_test_split(title, label, test_size=0.2)

In [ ]:
# import tokenizer & pad sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# mengubah kata ke bilangan numerik
tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(title_train) 
tokenizer.fit_on_texts(title_test)
 
# convert menjadi sequence
sequence_training = tokenizer.texts_to_sequences(title_train)
sequence_testing = tokenizer.texts_to_sequences(title_test)
 
padded_training = pad_sequences(sequence_training) 
padded_testing = pad_sequences(sequence_testing)

In [ ]:
# import tensorflow, keras, regularizers, layer, dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout

# embedding, lstm, model sequential
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(5, activation='softmax'),
])

# menggunakan loss categorical_crossentropy, optimizer adam (dengan learning rate 0.01)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy']
              )

In [ ]:
# menggunakan callback agar pengujian berhenti jika akurasi mencapai target
class callback(tf.keras.callbacks.Callback) :
  def on_epoch_end(self, epoch, logs={}) :
    if(logs.get('accuracy')) > 0.9 :
       if(logs.get('val_accuracy') > 0.8) :
         self.model.stop_training = True

In [ ]:
# train model dengan fit
model.fit(padded_training, 
                    label_train, 
                    batch_size=128, 
                    epochs=1000,
                    validation_data=(padded_testing, label_test), 
                    verbose=2, 
                    callbacks=callback()
                    )

Epoch 1/1000
14/14 - 0s - loss: 1.2919e-05 - accuracy: 1.0000 - val_loss: 4.4876 - val_accuracy: 0.7303
Epoch 2/1000
14/14 - 0s - loss: 1.8166e-06 - accuracy: 1.0000 - val_loss: 4.4900 - val_accuracy: 0.7281
Epoch 3/1000
14/14 - 0s - loss: 1.1093e-05 - accuracy: 1.0000 - val_loss: 4.4920 - val_accuracy: 0.7281
Epoch 4/1000
14/14 - 0s - loss: 4.0099e-05 - accuracy: 1.0000 - val_loss: 4.5035 - val_accuracy: 0.7281
Epoch 5/1000
14/14 - 0s - loss: 3.4819e-06 - accuracy: 1.0000 - val_loss: 4.5083 - val_accuracy: 0.7281
Epoch 6/1000
14/14 - 0s - loss: 2.5392e-06 - accuracy: 1.0000 - val_loss: 4.5108 - val_accuracy: 0.7281
Epoch 7/1000
14/14 - 0s - loss: 1.4080e-06 - accuracy: 1.0000 - val_loss: 4.5125 - val_accuracy: 0.7281
Epoch 8/1000
14/14 - 0s - loss: 1.5369e-06 - accuracy: 1.0000 - val_loss: 4.5135 - val_accuracy: 0.7281
Epoch 9/1000
14/14 - 0s - loss: 1.6535e-06 - accuracy: 1.0000 - val_loss: 4.5146 - val_accuracy: 0.7281
Epoch 10/1000
14/14 - 0s - loss: 7.7651e-06 - accuracy: 1.0000 -

KeyboardInterrupt: ignored